# Crypto OHLCV Data Exploration (TradingView-like)

This notebook loads cached CoinMarketCap parquet files from `data/cmc/` and renders interactive candlestick charts with volume using Plotly's dark theme.

In [ ]:
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
DATA_DIR = Path('../data/cmc')
SYMBOLS = ['btc', 'eth', 'sol', 'xrp', 'doge']
INTERVALS = ['hourly', 'daily']

def load_cached_ohlcv(symbol: str, interval: str) -> pd.DataFrame:
    file_path = DATA_DIR / f'{symbol}_{interval}.parquet'
    if not file_path.exists():
        raise FileNotFoundError(
            f'Missing file: {file_path}. Run the downloader script first: ' 
            'PYTHONPATH=src python scripts/fetch_cmc_ohlcv.py --years 3 --output-dir data/cmc'
        )

    df = pd.read_parquet(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    df = df.sort_values('timestamp').reset_index(drop=True)
    return df

available_files = sorted(p.name for p in DATA_DIR.glob('*.parquet'))
available_files

In [ ]:
SELECTED_SYMBOL = 'btc'
SELECTED_INTERVAL = 'hourly'  # 'hourly' or 'daily'

df = load_cached_ohlcv(SELECTED_SYMBOL, SELECTED_INTERVAL)
df.tail()

In [ ]:
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    row_heights=[0.75, 0.25],
    subplot_titles=[f'{SELECTED_SYMBOL.upper()} {SELECTED_INTERVAL.upper()} OHLC', 'Volume'],
)

fig.add_trace(
    go.Candlestick(
        x=df['timestamp'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Price',
        increasing_line_color='#26a69a',
        decreasing_line_color='#ef5350',
    ),
    row=1,
    col=1,
)

volume_colors = ['#26a69a' if c >= o else '#ef5350' for o, c in zip(df['open'], df['close'])]
fig.add_trace(
    go.Bar(x=df['timestamp'], y=df['volume'], marker_color=volume_colors, name='Volume'),
    row=2,
    col=1,
)

fig.update_layout(
    template='plotly_dark',
    height=850,
    xaxis_rangeslider_visible=False,
    hovermode='x unified',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(l=20, r=20, t=60, b=20),
)

fig.update_xaxes(showgrid=True, gridcolor='rgba(255,255,255,0.08)')
fig.update_yaxes(showgrid=True, gridcolor='rgba(255,255,255,0.08)')

fig.show()

In [ ]:
# Quick compare across symbols for close price (normalized).
frames = []
for sym in SYMBOLS:
    try:
        s = load_cached_ohlcv(sym, 'daily')[['timestamp', 'close']].copy()
        s['symbol'] = sym.upper()
        s['close_norm'] = s['close'] / s['close'].iloc[0]
        frames.append(s)
    except FileNotFoundError:
        pass

if not frames:
    raise FileNotFoundError('No cached daily parquet files found in data/cmc.')

compare = pd.concat(frames, ignore_index=True)
fig2 = go.Figure()
for sym, g in compare.groupby('symbol'):
    fig2.add_trace(go.Scatter(x=g['timestamp'], y=g['close_norm'], mode='lines', name=sym))

fig2.update_layout(
    template='plotly_dark',
    title='Normalized Daily Close Performance (Start = 1.0)',
    xaxis_title='Date',
    yaxis_title='Normalized Price',
    hovermode='x unified',
)
fig2.show()